In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load Data
df_domain1 = pd.read_json("/content/domain1_train_data.json", lines=True)
df_domain2 = pd.read_json("/content/domain2_train_data.json", lines=True)

# Label data for domain classification
df_domain1['domain_label'] = 0
df_domain2['domain_label'] = 1

# Undersampling Domain 2 to balance the dataset
df_domain2_undersampled = resample(df_domain2, replace=False, n_samples=len(df_domain1), random_state=123)

# Combine the datasets from both domains
df_combined = pd.concat([df_domain1, df_domain2_undersampled])

# Assume 'text' contains numerical data that can be directly scaled
# Pad sequences if necessary
max_length = 1982  # Choose based on your analysis
padded_data = pad_sequences(df_combined['text'].values, maxlen=max_length, padding='post')

# Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(padded_data)

# Split the data into training and testing sets
y = df_combined['domain_label'].values
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train a logistic regression model to classify domains
domain_classifier = LogisticRegression()
domain_classifier.fit(X_train, y_train)

# Evaluate the domain classifier
predictions = domain_classifier.predict(X_test)
print("Domain Classifier Accuracy:", accuracy_score(y_test, predictions))


Domain Classifier Accuracy: 0.56


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Load Data
df_domain1 = pd.read_json("/content/domain1_train_data.json", lines=True)
df_domain2 = pd.read_json("/content/domain2_train_data.json", lines=True)

# Label data for domain classification
df_domain1['domain_label'] = 0
df_domain2['domain_label'] = 1

# Combine the datasets from both domains
df_combined = pd.concat([df_domain1, df_domain2])

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df_combined['text'].apply(lambda x: ' '.join(map(str, x))))

# Split the data into training and testing sets
y = df_combined['domain_label'].values
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Train logistic regression classifier
logistic_classifier = LogisticRegression()
logistic_classifier.fit(X_train, y_train)

# Evaluate logistic regression classifier
logistic_predictions = logistic_classifier.predict(X_test)
logistic_accuracy = accuracy_score(y_test, logistic_predictions)
print("Logistic Regression Classifier Accuracy:", logistic_accuracy)


Logistic Regression Classifier Accuracy: 0.9933333333333333


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load data from JSON files
df_domain1 = pd.read_json("/content/domain1_train_data.json", lines=True)
df_domain2 = pd.read_json("/content/domain2_train_data.json", lines=True)

# Label data for domain classification
df_domain1['domain_label'] = 0
df_domain2['domain_label'] = 1

# Undersampling Domain 2 to balance the dataset
df_domain2_undersampled = resample(df_domain2, replace=False, n_samples=len(df_domain1), random_state=123)

# Combine the datasets from both domains
df_combined = pd.concat([df_domain1, df_domain2_undersampled])

# Convert the list of lists into a list of strings
X = [' '.join(map(str, text)) for text in df_combined['text']]

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

# Split the data into training and testing sets
y = df_combined['domain_label']
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Train SVM classifier
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)

# Predict on the test set
svm_predictions = svm_classifier.predict(X_test)

# Evaluate SVM classifier
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("SVM Classifier Accuracy:", svm_accuracy)


SVM Classifier Accuracy: 0.999


In [ ]:
import pandas as pd

# Load Data
df_domain1 = pd.read_json("/content/domain1_train_data.json", lines=True)
df_domain2 = pd.read_json("/content/domain2_train_data.json", lines=True)
df_test = pd.read_json("/content/test_data.json", lines=True)

# Assuming 'text' column contains lists of integers (sequence data)
def get_max_sequence_length(df):
    # Calculate the length of each sequence in the DataFrame
    return df['text'].apply(len).max()

# Get maximum lengths from each dataset
max_length_domain1 = get_max_sequence_length(df_domain1)
max_length_domain2 = get_max_sequence_length(df_domain2)
max_length_test = get_max_sequence_length(df_test)

# Find the overall maximum length
overall_max_length = max(max_length_domain1, max_length_domain2, max_length_test)

print("Maximum Sequence Length in Domain 1:", max_length_domain1)
print("Maximum Sequence Length in Domain 2:", max_length_domain2)
print("Maximum Sequence Length in Test Dataset:", max_length_test)
print("Overall Maximum Sequence Length Across All Datasets:", overall_max_length)


Maximum Sequence Length in Domain 1: 1982
Maximum Sequence Length in Domain 2: 1100
Maximum Sequence Length in Test Dataset: 1731
Overall Maximum Sequence Length Across All Datasets: 1982


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle


# Set the maximum length for padding
max_length_domain1 = 1200
X_seq_domain1 = pad_sequences(df_domain1['text'], maxlen=max_length_domain1, padding='post')
y_domain1 = df_domain1['label']
X_seq_domain1, y_domain1 = shuffle(X_seq_domain1, y_domain1, random_state=42)

# Split the data into training and testing sets
X_train_seq_domain1, X_test_seq_domain1, y_train_domain1, y_test_domain1 = train_test_split(X_seq_domain1, y_domain1, test_size=0.2, random_state=42)

# Further split training data for validation purposes
X_train_seq_base_domain1, X_val_seq_domain1, y_train_base_domain1, y_val_domain1 = train_test_split(X_train_seq_domain1, y_train_domain1, test_size=0.2, random_state=42)

# Define CNN models for Domain 1
domain1_models = {
    'cnn_model_1_domain1': Sequential([
        Embedding(input_dim=np.max(X_seq_domain1)+1, output_dim=50, input_length=max_length_domain1),
        Conv1D(32, 3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ]),
    'cnn_model_2_domain1': Sequential([
        Embedding(input_dim=np.max(X_seq_domain1)+1, output_dim=50, input_length=max_length_domain1),
        Conv1D(64, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ]),
    'cnn_model_3_domain1': Sequential([
        Embedding(input_dim=np.max(X_seq_domain1)+1, output_dim=50, input_length=max_length_domain1),
        Conv1D(64, 3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ])
}

# Compile models
for model in domain1_models.values():
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback to prevent overfitting
early_stopping_domain1 = EarlyStopping(monitor='val_loss', patience=1, verbose=1, restore_best_weights=True)

# Train models for Domain 1
for name, model in domain1_models.items():
    print(f"Training {name}...")
    model.fit(X_train_seq_base_domain1, y_train_base_domain1, epochs=10, batch_size=32, validation_data=(X_val_seq_domain1, y_val_domain1), callbacks=[early_stopping_domain1], verbose=1)

# Prepare data for SVM training
scaler_domain1 = StandardScaler()
X_train_flat_scaled_domain1 = scaler_domain1.fit_transform(X_train_seq_base_domain1.reshape(X_train_seq_base_domain1.shape[0], -1))
X_val_flat_scaled_domain1 = scaler_domain1.transform(X_val_seq_domain1.reshape(X_val_seq_domain1.shape[0], -1))
X_test_flat_scaled_domain1 = scaler_domain1.transform(X_test_seq_domain1.reshape(X_test_seq_domain1.shape[0], -1))

# Define and train SVM for Domain 1
svm_model_domain1 = SVC(kernel='rbf', probability=True)
svm_model_domain1.fit(X_train_flat_scaled_domain1, y_train_base_domain1)

# Collect predictions for meta learner
val_preds_domain1 = {name: model.predict(X_val_seq_domain1).flatten() for name, model in domain1_models.items()}
val_preds_domain1['svm_domain1'] = svm_model_domain1.predict_proba(X_val_flat_scaled_domain1)[:, 1]

test_preds_domain1 = {name: model.predict(X_test_seq_domain1).flatten() for name, model in domain1_models.items()}
test_preds_domain1['svm_domain1'] = svm_model_domain1.predict_proba(X_test_flat_scaled_domain1)[:, 1]

# Meta learner data preparation for Domain 1
meta_X_train_domain1 = np.column_stack(list(val_preds_domain1.values()))
meta_X_test_domain1 = np.column_stack(list(test_preds_domain1.values()))

# Train meta-learner for Domain 1
meta_learner_domain1 = LogisticRegression()
meta_learner_domain1.fit(meta_X_train_domain1, y_val_domain1)

# Meta-learner predictions and evaluation for Domain 1
meta_predictions_domain1 = meta_learner_domain1.predict(meta_X_test_domain1)
print("Meta Learner Ensemble Accuracy for Domain 1:", accuracy_score(y_test_domain1, meta_predictions_domain1))

# Save the meta learner model for Domain 1
# save_model(meta_learner_domain1, 'meta_learner_domain1_model.h5')



Training cnn_model_1_domain1...
Epoch 1/10
100/100 [==============================] - 12s 110ms/step - loss: 0.6869 - accuracy: 0.5753 - val_loss: 0.6793 - val_accuracy: 0.6450
Epoch 2/10
100/100 [==============================] - 9s 91ms/step - loss: 0.6398 - accuracy: 0.7825 - val_loss: 0.6493 - val_accuracy: 0.6850
Epoch 3/10
100/100 [==============================] - 10s 100ms/step - loss: 0.5453 - accuracy: 0.8475 - val_loss: 0.5988 - val_accuracy: 0.6888
Epoch 4/10
100/100 [==============================] - 10s 100ms/step - loss: 0.3994 - accuracy: 0.9156 - val_loss: 0.5456 - val_accuracy: 0.7200
Epoch 5/10
100/100 [==============================] - 10s 98ms/step - loss: 0.2492 - accuracy: 0.9628 - val_loss: 0.5166 - val_accuracy: 0.7450
Epoch 6/10
100/100 [==============================] - 9s 90ms/step - loss: 0.1407 - accuracy: 0.9894 - val_loss: 0.5112 - val_accuracy: 0.7487
Epoch 7/10
100/100 [==============================] - 10s 100ms/step - loss: 0.0786 - accuracy: 0.9956 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle, resample
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Assuming df_domain2 is already loaded
# Prepare sequences for Domain 2
max_length_domain2 = 1200
X_seq_domain2 = pad_sequences(df_domain2['text'], maxlen=max_length_domain2, padding='post')
y_domain2 = df_domain2['label']

# Balance the dataset by undersampling for Domain 2
def balance_data_domain2(X, y):
    # Combine into a DataFrame
    df_domain2 = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
    df_domain2['label'] = y

    # Separate majority and minority classes
    df_majority = df_domain2[df_domain2.label == df_domain2['label'].value_counts().idxmax()]
    df_minority = df_domain2[df_domain2.label == df_domain2['label'].value_counts().idxmin()]

    # Undersample majority class
    df_majority_undersampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=123)

    # Combine minority class with undersampled majority class
    df_balanced_domain2 = pd.concat([df_majority_undersampled, df_minority])

    # Shuffle to avoid ordering bias
    df_balanced_domain2 = shuffle(df_balanced_domain2, random_state=42)

    return df_balanced_domain2.iloc[:, :-1].values, df_balanced_domain2['label'].values

# Balance and split the data for Domain 2
X_balanced_domain2, y_balanced_domain2 = balance_data_domain2(X_seq_domain2, y_domain2)
X_train_seq_domain2, X_test_seq_domain2, y_train_domain2, y_test_domain2 = train_test_split(X_balanced_domain2, y_balanced_domain2, test_size=0.2, random_state=42)
X_train_seq_base_domain2, X_val_seq_domain2, y_train_base_domain2, y_val_domain2 = train_test_split(X_train_seq_domain2, y_train_domain2, test_size=0.2, random_state=42)

# Define CNN models for Domain 2
domain2_models = {
    'cnn_model_1_domain2': Sequential([
        Embedding(input_dim=np.max(X_seq_domain2)+1, output_dim=50, input_length=max_length_domain2),
        Conv1D(32, 3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ]),
    'cnn_model_2_domain2': Sequential([
        Embedding(input_dim=np.max(X_seq_domain2)+1, output_dim=50, input_length=max_length_domain2),
        Conv1D(64, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ]),
    'cnn_model_3_domain2': Sequential([
        Embedding(input_dim=np.max(X_seq_domain2)+1, output_dim=50, input_length=max_length_domain2),
        Conv1D(64, 3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ])
}

# Compile and train each model for Domain 2
early_stopping_domain2 = EarlyStopping(monitor='val_loss', patience=1, verbose=1, restore_best_weights=True)
for name, model in domain2_models.items():
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print(f"Training {name}...")
    model.fit(X_train_seq_base_domain2, y_train_base_domain2, epochs=10, batch_size=32, validation_data=(X_val_seq_domain2, y_val_domain2), callbacks=[early_stopping_domain2], verbose=1)

# SVM training for balanced data in Domain 2
scaler_domain2 = StandardScaler()
X_train_flat_scaled_domain2 = scaler_domain2.fit_transform(X_train_seq_base_domain2.reshape(X_train_seq_base_domain2.shape[0], -1))
X_val_flat_scaled_domain2 = scaler_domain2.transform(X_val_seq_domain2.reshape(X_val_seq_domain2.shape[0], -1))
X_test_flat_scaled_domain2 = scaler_domain2.transform(X_test_seq_domain2.reshape(X_test_seq_domain2.shape[0], -1))

svm_model_domain2 = SVC(kernel='rbf', probability=True)
svm_model_domain2.fit(X_train_flat_scaled_domain2, y_train_base_domain2)

# Collect predictions for meta learner in Domain 2
val_preds_domain2 = {name: model.predict(X_val_seq_domain2).flatten() for name, model in domain2_models.items()}
val_preds_domain2['svm_domain2'] = svm_model_domain2.predict_proba(X_val_flat_scaled_domain2)[:, 1]

test_preds_domain2 = {name: model.predict(X_test_seq_domain2).flatten() for name, model in domain2_models.items()}
test_preds_domain2['svm_domain2'] = svm_model_domain2.predict_proba(X_test_flat_scaled_domain2)[:, 1]

# Meta learner for Domain 2
meta_X_train_domain2 = np.column_stack(list(val_preds_domain2.values()))
meta_X_test_domain2 = np.column_stack(list(test_preds_domain2.values()))

meta_learner_domain2 = LogisticRegression()
meta_learner_domain2.fit(meta_X_train_domain2, y_val_domain2)

meta_predictions_domain2 = meta_learner_domain2.predict(meta_X_test_domain2)
print("Meta Learner Ensemble Accuracy for Domain 2:", accuracy_score(y_test_domain2, meta_predictions_domain2))


Training cnn_model_1_domain2...
Epoch 1/10
60/60 [==============================] - 8s 124ms/step - loss: 0.6853 - accuracy: 0.5594 - val_loss: 0.6719 - val_accuracy: 0.7396
Epoch 2/10
60/60 [==============================] - 6s 103ms/step - loss: 0.6314 - accuracy: 0.7964 - val_loss: 0.6335 - val_accuracy: 0.7375
Epoch 3/10
60/60 [==============================] - 7s 113ms/step - loss: 0.5474 - accuracy: 0.8453 - val_loss: 0.5820 - val_accuracy: 0.7542
Epoch 4/10
60/60 [==============================] - 6s 102ms/step - loss: 0.4402 - accuracy: 0.8802 - val_loss: 0.5347 - val_accuracy: 0.7688
Epoch 5/10
60/60 [==============================] - 7s 116ms/step - loss: 0.3264 - accuracy: 0.9339 - val_loss: 0.4928 - val_accuracy: 0.7917
Epoch 6/10
60/60 [==============================] - 7s 109ms/step - loss: 0.2179 - accuracy: 0.9781 - val_loss: 0.4588 - val_accuracy: 0.8021
Epoch 7/10
60/60 [==============================] - 7s 110ms/step - loss: 0.1348 - accuracy: 0.9937 - val_loss: 0.43

In [ ]:
# import numpy as np
# import pandas as pd
# from tensorflow.keras.models import load_model
# import joblib
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from sklearn.preprocessing import StandardScaler

# # Ensure the test data is appropriately preprocessed and padded
# test_data = pd.read_json("/content/test_data.json", lines=True)
# X_test_padded = pad_sequences(test_data['text'], maxlen=1200, padding='post')
# X_test = vectorizer.transform(test_data['text'].apply(lambda x: ' '.join(map(str, x))))

# # Domain classification
# domain_predictions = domain_classifier.predict(X_test)
# print(domain_predictions)

# # Initialize lists for storing predictions
# final_predictions = []

# # Process each test instance according to the predicted domain
# for idx, domain in enumerate(domain_predictions):
#     if domain == 0:  # Domain 1
#         model_preds = {name: model.predict(X_test_padded[idx:idx+1]).flatten() for name, model in domain1_models.items()}
#         model_preds['svm'] = svm_model_domain1.predict_proba(X_test_padded[idx:idx+1])[:, 1]
#         meta_input = np.column_stack(list(model_preds.values()))
#         final_prediction = meta_learner_domain1.predict(meta_input)
#     else:  # Domain 2
#         model_preds = {name: model.predict(X_test_padded[idx:idx+1]).flatten() for name, model in domain2_models.items()}
#         model_preds['svm'] = svm_model_domain2.predict_proba(X_test_padded[idx:idx+1])[:, 1]
#         meta_input = np.column_stack(list(model_preds.values()))
#         final_prediction = meta_learner_domain2.predict(meta_input)

#     final_predictions.append(final_prediction[0])

# # Output final predictions
# final_predictions = np.array(final_predictions)
# print("Final Predictions:", final_predictions)

import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler

# Ensure the test data is appropriately preprocessed and padded
test_data = pd.read_json("/content/test_data.json", lines=True)
X_test_padded = pad_sequences(test_data['text'], maxlen=1200, padding='post')
X_test = vectorizer.transform(test_data['text'].apply(lambda x: ' '.join(map(str, x))))

# Domain classification
domain_predictions = domain_classifier.predict(X_test)
print(domain_predictions)

# Initialize lists for storing predictions
final_predictions = []
counter = 0  # Initialize the counter

# Process each test instance according to the predicted domain
for idx, domain in enumerate(domain_predictions):
    counter += 1  # Increment the counter for each test instance processed
    if domain == 0:  # Domain 1
        model_preds = {name: model.predict(X_test_padded[idx:idx+1]).flatten() for name, model in domain1_models.items()}
        model_preds['svm'] = svm_model_domain1.predict_proba(X_test_padded[idx:idx+1])[:, 1]
        meta_input = np.column_stack(list(model_preds.values()))
        final_prediction = meta_learner_domain1.predict(meta_input)
    else:  # Domain 2
        model_preds = {name: model.predict(X_test_padded[idx:idx+1]).flatten() for name, model in domain2_models.items()}
        model_preds['svm'] = svm_model_domain2.predict_proba(X_test_padded[idx:idx+1])[:, 1]
        meta_input = np.column_stack(list(model_preds.values()))
        final_prediction = meta_learner_domain2.predict(meta_input)

    final_predictions.append(final_prediction[0])
    print(f"Processed instance {counter}: Domain {domain}, Prediction {final_prediction[0]}")

# Output final predictions
final_predictions = np.array(final_predictions)
print("Final Predictions:", final_predictions)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 26ms/step
Processed instance 2751: Domain 1, Prediction 0
1/1 [==============================] - 0s 29ms/step
Processed instance 2752: Domain 1, Prediction 1
1/1 [==============================] - 0s 25ms/step
Processed instance 2753: Domain 1, Prediction 1
1/1 [==============================] - 0s 25ms/step
Processed instance 2754: Domain 1, Prediction 1
1/1 [==============================] - 0s 29ms/step
Processed instance 2755: Domain 1, Prediction 0
1/1 [==============================] - 0s 31ms/step
Processed instance 2756: Domain 0, Prediction 0
1/1 [==============================] - 0s 30ms/step
Processed instance 2757: Domain 1, Prediction 1
1/1 [==============================] - 0s 28ms/step
Processed instance 2758: Domain 0, Prediction 1
1/1 [==============================] - 0s 27ms/step
Processed instance 2759: Domain 0, Prediction 1
1/1 [==============================] - 0s 28ms/st

In [ ]:
predictions_df = pd.DataFrame(final_predictions, columns=['Predictions'])
predictions_df.to_csv('/content/final_predictions.csv', index=False)